In [39]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
#from datetime import date
from scipy.stats import norm
from scipy.optimize import minimize
from scipy import interpolate
from scipy.optimize import fsolve
from dateutil.relativedelta import *
import scipy.integrate as integrate
from scipy.interpolate import interp1d
import warnings
import numdifftools as nd
from dateutil.relativedelta import relativedelta
#warnings.filterwarnings('ignore')
%matplotlib inline

### fit default and prepayment models

In [41]:
#fit dafualt model, get two set of params for two pools: FIXparam, and ARMparam
FIXdata = pd.read_csv('FIX_perf_s.csv',header=None,names=['ID','age','spread2','spread','LTV','Rbala','summer','default','prepay'])
FIXdata['spread2'] = FIXdata['spread2'] / 100
FIXdata['']
ARMdata = pd.read_csv('ARM_perf_s.csv',header=None,names=['ID','age','spread2','spread','LTV','Rbala','summer','default','prepay'])

def log_log_like(param, tb, te, event, covars):
    """
    param[0]: g
    param[1]: p
    param[2:]: betas
    """
    assert len(param) == len(covars) + 2
    n_coef = len(covars)
    g, p, coef = param[0], param[1], np.array(param[2:])
    coef_prod = coef.dot(covars)
    sum_1_array = np.log(g*p) + (p-1) * np.log(g*te) - np.log(1+np.power(g*te, p)) + coef_prod
    sum_2_array = np.exp(coef_prod) * (np.log(1+np.power(g*te, p)) - np.log(1+np.power(g*tb, p)))
    return -(sum_1_array[event == 1].sum() - sum_2_array.sum())

#fit 4 models for two pools
#covars are spread squared, spread, LTV, Rbala, summer
#FRM default
te = FIXdata['age'].values
tb = np.insert(te[:-1], 0, 0)
default = FIXdata['default'].values
covars = np.array(np.transpose(FIXdata.iloc[:,2:7].values))
FRMDefaultres = minimize(log_log_like, [0.1,0.1,0.1,0.1,0.1,0.1,0.1],args=(tb,te,default,covars), method='Nelder-Mead', options={'xtol': 1e-8, 'disp': False})
FRMDefaultParam = FRMDefaultres.x
#FRM prepay
te = FIXdata['age'].values
tb = np.insert(te[:-1], 0, 0)
prepay = FIXdata['prepay'].values
covars = np.array(np.transpose(FIXdata.iloc[:,2:7].values))
FRMPrepayres = minimize(log_log_like, [0.1,0.1,0.1,0.1,0.1,0.1,0.1],args=(tb,te,prepay,covars), method='Nelder-Mead', options={'xtol': 1e-8, 'disp': False})
FRMPrepayParam = FRMPrepayres.x

#ARM default
te = ARMdata['age'].values
tb = np.insert(te[:-1], 0, 0)
default = ARMdata['default'].values
covars = np.array(np.transpose(ARMdata.iloc[:,2:7].values))
ARMDefaultres = minimize(log_log_like, [0.1,0.1,0.1,0.1,0.1,0.1,0.1],args=(tb,te,default,covars), method='Nelder-Mead', options={'xtol': 1e-8, 'disp': False})
ARMDefaultParam = ARMDefaultres.x
#ARM prepay
te = ARMdata['age'].values
tb = np.insert(te[:-1], 0, 0)
prepay = ARMdata['prepay'].values
covars = np.array(np.transpose(ARMdata.iloc[:,2:7].values))
ARMPrepayres = minimize(log_log_like, [0.1,0.1,0.1,0.1,0.1,0.1,0.1],args=(tb,te,prepay,covars), method='Nelder-Mead', options={'xtol': 1e-8, 'disp': False})
ARMPrepayParam = ARMPrepayres.x




/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app
/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in power
  app.launch_new_instance()
/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in power
  from ipykernel import kernelapp as app
/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in power
  app.launch_new_instance()
/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()
/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()
/Users/zhengy

In [44]:
print(FRMDefaultParam, FRMPrepayParam, ARMDefaultParam, ARMPrepayParam)

[  5.77202386e-02   4.68403727e+02  -2.74288290e+02   1.23086773e+02
  -5.24568930e+02   1.04169530e+03  -7.00669685e+01] [ 184.85318418   80.88769345  -49.55340758  172.4319948  -352.8553428
  138.27965586  333.51743342] [  4.95585974e-02   3.12014632e+02  -1.57168746e+02  -3.07883353e+02
   8.43711087e+02   5.06774840e+01   8.52932442e+00] [  7.05335011e-02   3.59940555e+02   3.91955573e+01  -2.12234624e+02
   8.47863047e+02  -4.00104863e+02  -1.41317207e+01]


### SMM functions

In [95]:
def calcHousePrice(Z, r, H0, q=0.025, theta = 0.12):
    # Z - MC factors, n paths, m time stps
    # r - array of short rate, q - rental flow rate, theta = volatility
    # return an array of house prises over time
    n,m = Z.shape
    t = np.array([np.arange(1/12,m/12,1/12)]*n)
    H = H0 * np.exp((r-q-0.5*theta**2)*t+theta*Z.cumsum(axis=0))
    #print('Z',Z.cumsum(axis=0).shape)
    H = np.c_[[H0]*n,H]
    #print(H.shape)
    return H #return a n x m matrix

def calcLTV(Rbala, H):
    #Rbala - remaining balance, H - house price
    #return LTV
    return Rbala / H

#covars are spread, spread squared, LTV, Rbala, summer
#params  = g, p, coef
def calcLam(t,spread,Rbala,HousePrice,summer,params):
    #print(spread**2, spread, calcLTV(Rbala, HousePrice), Rbala, summer)
    covars = np.array([spread**2, spread, calcLTV(Rbala, HousePrice), Rbala, summer])

    g,p,betas = params[0],params[1],params[2:]
    lam = g*p*pow(g*t,p-1)/(1+pow(g*t,p))*np.exp(betas.dot(covars))
    print(t,betas)
    print(covars.shape,lam)
    print(np.exp(betas.dot(covars)))
    return lam

def calcSMM(t,spread,Rbala,HousePrice,summer,params):
    lam = calcLam(t,spread,Rbala,HousePrice,summer,params)
    SMM = 1 - np.exp(-12*lam*dt_smm[0,t])
    return SMM, lam

### fit Hull White model

In [46]:
libor_df = pd.read_excel('bbg_rates_data.xlsx', sheetname=0)
caps_df = pd.read_excel('bbg_rates_data.xlsx', sheetname=2)

# Discount factors
dates = pd.to_datetime(libor_df['Date'].values)
forward_rates = libor_df.iloc[:,3]/100.0
n = forward_rates.size
taus = (dates[1:] - dates[:-1]).days/360.0
discount_factors= np.r_[1,1/np.cumprod(forward_rates[:-1]*taus+1)]

expiries = caps_df['Expiry']
expiries = np.array(list(int(e[:-2]) for e in expiries))
flat_vols = caps_df.iloc[:,1]/100
strikes = caps_df.iloc[:,2]/100
val_date = dates[0]
sett_dates = pd.to_datetime(libor_df.iloc[1:,0])
mats = list(d.days/365.0 for d in (sett_dates - val_date))

def calc_blk_cap(v, n, x, notional=10000000): 
    """
    Calculate market cap price
    v: market cap vol
    n: total number of caplets
    x: ATM strike
    """
    z = discount_factors[2:2+n]
    f = forward_rates[1:1+n]
    vol = v*np.sqrt(mats[:n])
    d1 = np.log(f / x) / vol + 0.5 * vol
    d2 = d1 - vol
    return sum((norm.cdf(d1) * f - x * norm.cdf(d2)) * z * taus[1:n+1] * notional)
vcalc_blk_cap = np.vectorize(calc_blk_cap)
blk_caps = vcalc_blk_cap(flat_vols,4*expiries-1,strikes)
blk_caps

caplet_expiry = (pd.to_datetime(libor_df.iloc[1:,0].values)-val_date).days/365
def B_HW(kappa,T): # Hull-White B(T) fucntion
    return  (1 - np.exp(-kappa * T)) / kappa

def calc_hw_cap(sigma, kappa, n, x, notional=10000000):
    """
    calculate hull-white cap price
    n: total number of caplets
    x: ATM strike
    """
    B = B_HW(kappa,np.diff(caplet_expiry[:n+1])) 
    Ti_minus_1 = caplet_expiry[:n]
    Sz = B * sigma * np.sqrt(B_HW(2*kappa,Ti_minus_1))
    K = 1+x*taus[1:1+n]
    df2 = discount_factors[2:2+n]
    df1 = discount_factors[1:1+n]
    d1 = np.log(df2/df1 * K) / Sz + 0.5 * Sz
    d2 = d1 - Sz
    return sum(notional * (norm.cdf(-d2) * df1 - K * norm.cdf(-d1)* df2))
vcalc_hw_cap = np.vectorize(calc_hw_cap)

In [47]:
def calibration_object(param):
    sigma, kappa = param
    hw_caps =vcalc_hw_cap(sigma, kappa, expiries*4-1, strikes)
    error = np.sum((blk_caps - hw_caps)**2)
    return error
res = minimize(calibration_object, [0.1,0.1], method='nelder-mead', options={'xtol': 1e-10, 'disp': False})
sigma, kappa = res.x
print(sigma, kappa)

0.0153084797541 0.0805385240171


In [48]:
tck = interpolate.splrep(dates, discount_factors, s=0)
month_starts = pd.date_range(start=dates[0], end=dates[-1], freq='MS')
ms_i8 = month_starts.view('i8')
monthly_discount_factors = interpolate.splev(ms_i8, tck, der=0)
ts = (month_starts - dates[0])/ np.timedelta64(1, 'D')/365.0
fm = -np.diff(np.log(monthly_discount_factors)) /np.diff(ts) 
dt = np.diff(month_starts)/ np.timedelta64(1, 'D')/365.0
df = np.diff(fm) / dt[:-1]
thetas = df + kappa * fm[:-1] + sigma**2 /2/kappa *(1 - np.exp(-2*kappa*ts[:-2]))

In [49]:
n = 20000 #MC paths
m = 354 - 12*3-3# number of Timestep each path 
PoolDates =  pd.date_range(datetime(2009,6,30), end=datetime(2036,3,27),closed='left', freq='MS') # 1st day of the month
PaymentDates =PoolDates+ timedelta(days=14) # 15 days of the month
t_thetas = (month_starts[:-2]- val_date).days/365 # thetas corresponding time
tpool = (PoolDates- val_date).days/365 # pool date act/365
dt = np.diff(tpool)
tp = (PaymentDates- val_date).days/365 # payment date act/365
dtp=np.r_[tp[0],np.diff(tp)]
dt_smm = np.r_[dt, dt[-1]].reshape(1,-1)
np.random.seed(123)
Z = np.random.normal(size=[int(n/2),m])
Z = np.r_[Z,-Z] #Anthithetic Variable

def DF_HW_MC(sigma,kappa,Z,r0=fm[0],s=0,dt=dt,dtp=dtp):# assume r0 = fm[0]
    """
    dt: we generate short rate 1st date each month
    dtp: we discount short rate 15th date each month
    s: OAS spread, default is 0
    """
    n = Z.shape[0]
    r = np.c_[r0*np.ones(n),np.zeros([n,m-1])] 
    for i in range(m-1):
        r[:,i+1]= thetas[i]*dt[i]+sigma*np.sqrt(dt[i])*Z[:,i]+(1-kappa*dt[i])*r[:,i]
    discount_factors_MC = np.exp(-((r+s)*dtp[:m].reshape(1,-1)).cumsum(axis=1))
    return r,discount_factors_MC

#calculate 10yr rate: 
def A(t,T,T_theta=t_thetas,thetas=thetas,k=kappa,sigma_=sigma):
    """
    Hull-white model A(t,T)
    T_theta: corresponding time of theta, it has the same dimension with thetas
    """
    b = B_HW(k,T-t)
    # rectangle approximation of the integral from t to T
    x = (T_theta>=t)&(T_theta<T)
    ts = T_theta[x]
    dt = np.r_[np.diff(ts),T-ts[-1]] 
    a = -np.dot(thetas[x]*dt,B_HW(k,T-ts)) +sigma_**2/2/k**2*(T-t+ B_HW(2*k,T-t)-2*b)
    return a

#!!!!!!!!!!!!!!
vA = np.vectorize(A,excluded=[2,3,4,5])
#!!!!!!!!!!!!!!

def r_spot(t,tau,rt,T_theta=t_thetas,thetas=thetas,k=kappa):
    """
    Hull white spot rate from t to t+tau
    """
    return  (-(vA(t,t+tau)/tau))+(1/tau)* B_HW(k,tau)*rt 


fixings = [0.0520, 0.0508, 0.0495] # 10 year libor
# lag = 3 # 3-month lag
# tau = 10 # 10-year libor
def r_spot_10(r_MC,tau=10,lag=3, fixings=fixings, T_theta=t_thetas,thetas=thetas,k=kappa):
    n,m = r_MC.shape
    fixings = fixings*np.ones(n).reshape(-1,1)
    yield10yr = np.c_[fixings,r_spot(T_theta[:m-lag],tau,r_MC[:,:-lag],T_theta,thetas,k)]
    return yield10yr

### Cash flow calculation

In [50]:
df_curve = pd.read_excel('USD23_Curve.xlsx', sheetname='Worksheet')
df_cap=pd.read_excel('USD_Cap_VOL.xlsx', sheetname='Quotes')
df_vol=pd.read_excel('USD_Cap_VOL.xlsx', sheetname='Sheet1')
#coupon gap has unit of bps, converted to unit of 1

In [51]:
def pmt(Bal,rate,nper):
    a = (1+rate)**(nper)
    return Bal *rate*a/(a-1) if nper>0 else 0
vpmt = np.vectorize(pmt)
def RunningBalance (Remain,Cur,eps = 0):
    return Cur if Remain-Cur>eps else Remain
vRunningBalance = np.vectorize(RunningBalance)

In [88]:
def Loan_cf(start_time,HousePrices,LoanPricipal,n,m,s,rec_rate=0.4,IsFixed=True,Libor=0,pc=0,lol=0,eps=0,LiborLag=3):
    """
    start_time: keey track of the month of each iteration, then determine summer indicator
    Libor: one month Libor rate,dimension = n * (m+1), 
           note that we need Libor last period for floating rate,the first Libor rate is Apr 30, 2009.
    LoanPricipal: Loan Principal
    n: MC paths
    m: loan Periods, WARM, 354
    IsFixed: True / False. If true, compute FRM; if false, compute ARM
    S:IsFixed=True, FixedCoupon; IsFixed=False, Spread on Libor
    pc: Periodic cap
    lol:LOL Cap
    rec_rate: recover rate, scalar
    Liborlag: for coupon gap adjustment
    ###############
    dependent variables:
    SSM: for prepayment,dimension = n * m
    def_rate: default prob each period, dimension = n * m
    ##############
    """
    global FRMDefaultParam, FRMPrepayParam, ARMDefaultParam, ARMPrepayParam
    
    AMTp = np.zeros([n,m]) # amortization of pool
    Intp = np.zeros([n,m]) #interest of pool
    Prp = np.zeros([n,m]) #principal of pool
    PPCf = np.zeros([n,m]) #Prepay CF
    Defaultp = np.zeros([n,m]) #Default CF
    per = np.arange(m) #period
    Balp = np.zeros([n,m+1]) # balance of pool
    Balp[:,0] = LoanPricipal #Initial balance
    SMM = np.zeros([n,m]) # prepayment rate
    def_rate = np.zeros([n,m]) #default rate
    
    current_date = start_time
    current_month = current_date.month
    for i in range(m):
        summer = 1 if current_month in [5,6,7,8] else 0
        current_date = (current_date + relativedelta(month = 1))
        current_month = current_date.month

        if IsFixed:
            pool_r = s # scalar
            PrepayParam = FRMPrepayParam
            DefaultParam = FRMDefaultParam
            spread = pool_r - Libor[:,i+LiborLag]
        else: 
            pool_r = np.minimum(np.maximum(np.minimum(Libor[:,i+LiborLag]+s,Libor[:,i+LiborLag-1]+pc),Libor[:,i+LiborLag-1]-pc),lol)
            PrepayParam = ARMPrepayParam
            DefaultParam = ARMDefaultParam
            spread = pool_r - Libor[:,i+LiborLag]
            
        pool_r = pool_r/12
        AMTp[:, i] = vpmt(Balp[:, i], pool_r, m - per[i])  # amortization amount
        Intp[:, i] = Balp[:, i] * pool_r  # interest
        Prp[:, i] = vRunningBalance(Balp[:, i], AMTp[:, i] - Intp[:, i], eps)  # principal
        
        
        SMM[:,i],lam = calcSMM(i,spread,Balp[:, i],HousePrices[:,i],summer,PrepayParam)
        PPCf[:, i] = (Balp[:, i] - Prp[:, i]) * SMM[:, i]  # prepay cf        
    
        def_rate[:,i],lam = calcSMM(i,spread,Balp[:, i],HousePrices[:,i],summer,DefaultParam)
        Defaultp[:,i] = (Balp[:, i] - Prp[:, i] - PPCf[:, i])*(1-rec_rate)*def_rate[:,i]
        Balp[:, i + 1] = Balp[:, i] - Prp[:, i] - PPCf[:, i]-Defaultp[:,i]  # update balance
        
    return [AMTp, Intp, Prp, PPCf, Balp, Defaultp]

In [85]:
# Waterfall Distribution of principal and interest

def calc_total_CF(Prp, PPCf, Intp, Balp,Defaultp,Libor,
                  OTCRate1=.031,OTCRate2=.062,OTCCap=3967158,
                  InitialLoan = [79036000, 714395000],
                  beginning_balpc = [0,107769,24954,38481,30150,18646,16265,15075,13488,13092,619,0,0,0],
                  LiborSpread = [0.08,0.18,0.28,0.36,0.38,0.39,0.51,0.55,0.62,1.15,1.4,2.25,2.25,0],LiborLag=3):
    """
    Prp: Pool principal, dimension = n * m
    PPCf: Pool prepayment, dimension = n * m
    Intp: Pool Interest, dimension = n * m
    Balp: Pool Balance,dimension = n * (m+1)
    Defaultp: pool default amount, dimension = n * m
    InitialLoan: Mortgage Initial Balance, dimension = npool*1
    Libor: one month Libor rate,dimension = n * (m+1), 
           note that we need Libor last period for floating rate,the first Libor rate is Apr 30, 2009.
    
    "Overcollateralization Target Amount” Input:
    OTCRate1: %of the aggregate Stated Principal Balance of the mortgage loans at the deal origination date, 3.1%
    OTCRate2:the current aggregate Stated Principal Balance of the mortgage loans at time period t, 6.2%
    OTCCap: 3,967,158
    beginning_balpc: Current Principal of CMO, the order is ['A1','A2','A3','M1','M2','M3','M4','M5','M6','M7','M8','M9','M10','CE']
    """
    
    #intial configue
    n,m = Prp.shape 
    nclass = len(beginning_balpc)
    Prpc = np.zeros([nclass,n,m])
    Balpc = np.zeros([nclass,n,m+1])
    IntPC = np.zeros([nclass,n,m])
    Balpc[:,:,0] = np.tile(beginning_balpc,(n,1)).T
    PrTrach = Prp + PPCf
    InitialOTA = OTCRate1 * sum(InitialLoan)
    # waterfall distribution
    for k in range(m): 
        DisPr = 0 # running pricipal that has distributed
        DisInt = 0 # running Interest that has distributed
        for cl_i in range(nclass-1): #exclude CE class
            Prpc[cl_i,:,k]=np.maximum(0,np.minimum(Balpc[cl_i,:,k],PrTrach[:,k]-DisPr))
            IntPC[cl_i,:,k]=np.minimum(Intp[:,k],Balpc[cl_i,:,k]*(Libor[:,k+LiborLag]+LiborSpread[cl_i]))
            Balpc[cl_i,:,k+1] = np.maximum(0,Balpc[cl_i,:,k]-Prpc[cl_i,:,k])
            DisPr += Prpc[cl_i,:,k]
            DisInt+= IntPC[cl_i,:,k]
        
        #Default payment
        ExSpread = Intp[:,k]-IntPC[:,:,k].sum(axis=0)  #Excess Spread
        OCA = Balp[:,k+1] - Balpc[:-1,:,k+1].sum(axis=0) #Overcollateralization Amount, minus part exclude CE class balance
        RemainDefault = np.maximum(Defaultp[:,k]-ExSpread,0)
        ExSpread -= (Defaultp[:,k]-RemainDefault)
        CurrentDefault = RemainDefault
        RemainDefault = np.maximum(RemainDefault-OCA,0)
        OCA -= (CurrentDefault-RemainDefault)
        CurrentDefault = RemainDefault
        #print(CurrentDefault)
        cl_i = nclass - 1 # start from CE class
        while cl_i>=0 and (CurrentDefault>0).all():
            RemainDefault = np.maximum(RemainDefault-Balpc[cl_i,:,k+1],0)
            Balpc[cl_i,:,k+1] -= (CurrentDefault-RemainDefault)
            CurrentDefault = RemainDefault
            cl_i-=1
            
        #Extra Principal distribute
        OTA = np.maximum(OTCCap,np.maximum(InitialOTA,np.minimum(InitialOTA,OTCRate2*Balp[:,k+1])))#Overcollateralization Target Amount
        ExPr = np.minimum(np.maximum(OTA-OCA,0),ExSpread) #Extra Principal Distribution Amount
        cl_i = 0
        CurrExPr,RemainExPr = ExPr,ExPr
        while cl_i<nclass and (CurrExPr>0).all():
            RemainExPr = np.maximum(RemainExPr-Balpc[cl_i,:,k+1],0) #Distribute
            Prpc[cl_i,:,k] +=(CurrExPr-RemainExPr)
            Balpc[cl_i,:,k+1] -= (CurrExPr-RemainExPr)
            CurrExPr = RemainExPr
            cl_i+=1
            
    TotalCF = IntPC+Prpc
    return TotalCF

In [96]:
# Hazard functions: use four sets of parameters:
#FRMDefaultparam, FRMPrepayparam, ARMDefaultparam, ARMPrepayparam

#test functions - ARM
H0 = 52416155 / 0.856
r0 = fm[0]
s = 0
r_MC, discount_factors_MC = DF_HW_MC(sigma,kappa,Z,r0,s,dt)
HousePrices = calcHousePrice(Z,r_MC,H0) # the output includes H0

start_time = pd.to_datetime('2009-06-30')
s = 0.07419
Libor=r_MC
FRM = Loan_cf(start_time,HousePrices,H0,n,m,s,rec_rate=0.4,IsFixed=True,Libor=r_MC,LiborLag=0)

#print((r_MC-0.025).shape)


/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in exp
/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply
/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: overflow encountered in exp


0 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ inf  inf  inf ...,  inf  inf  inf]
0 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ inf  inf  inf ...,  inf  inf  inf]
1 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
1 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
2 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
2 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
3 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.517

29 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
29 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
30 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
30 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
31 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
31 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
32 [ -49.55340758  172.4319948  -352.8553428   138.27965586  

/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in double_scalars
/Users/zhengyanghe/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


35 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
35 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
36 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
36 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
37 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
37 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
38 [ -49.55340758  172.4319948  -352.8553428   138.27965586  

...

180 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
180 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
181 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
181 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
182 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
182 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
183 [ -49.55340758  172.4319948  -352.8553428   138.279

210 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
210 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
211 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
211 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
212 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
212 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
213 [ -49.55340758  172.4319948  -352.8553428   138.279

238 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
238 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
239 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
239 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
240 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
240 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
241 [ -49.55340758  172.4319948  -352.8553428   138.279

268 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
268 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
269 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
269 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
270 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
270 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
271 [ -49.55340758  172.4319948  -352.8553428   138.279

298 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
298 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
299 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
299 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
300 [ -49.55340758  172.4319948  -352.8553428   138.27965586  333.51743342]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
300 [ -274.28829038   123.08677304  -524.56892952  1041.69529864   -70.06696851]
(5,) [ nan  nan  nan ...,  nan  nan  nan]
[ nan  nan  nan ...,  nan  nan  nan]
301 [ -49.55340758  172.4319948  -352.8553428   138.279

In [66]:
print(FRM)

[array([[ 442000.67244011,              nan,              nan, ...,
                     nan,              nan,              nan],
       [ 442000.67244011,              nan,              nan, ...,
                     nan,              nan,              nan],
       [ 442000.67244011,              nan,              nan, ...,
                     nan,              nan,              nan],
       ..., 
       [ 442000.67244011,              nan,              nan, ...,
                     nan,              nan,              nan],
       [ 442000.67244011,              nan,              nan, ...,
                     nan,              nan,              nan],
       [ 442000.67244011,              nan,              nan, ...,
                     nan,              nan,              nan]]), array([[ 378578.12884054,              nan,              nan, ...,
                     nan,              nan,              nan],
       [ 378578.12884054,              nan,              nan, ...,
       

In [ ]:
a = np.zeros([3,3])
print(a)
#b = np.array([a]*3)
a[:,1] = np.array([1,2,3])
print(a)

In [78]:
te = FIXdata['age'].values
tb = np.insert(te[:-1], 0, 0)
print(type(te[0]))
print(type(tb[0]))

<class 'numpy.int64'>
<class 'numpy.int64'>
